In [1]:
# Script demonstrating pruning of a Fully Connected layer with no bias.

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pdb
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if torch.cuda.is_available():
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [4]:
def train(log_interval, model, device, train_loader, optimizer, epoch, alpha =1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, alpha)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}, alpha: {}, [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, alpha, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader, alpha=1):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data, alpha)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Alpha {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(alpha,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

32768it [00:02, 12643.13it/s]            
1654784it [00:02, 748890.60it/s]                            
8192it [00:01, 7217.05it/s]             


In [5]:
# Original Pre-trained model (Trained using mnist.py)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 1 * 1, 128, bias =False)
        self.fc2 = nn.Linear(128, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
model = Net().to(device)

In [6]:
# Load weights
PATH = 'mnist_cnn_nobias.pt'
model_dict= torch.load(PATH, map_location = device)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [7]:
# Test to see if the weights of the pretrained model have been correctly loaded
test(model, device, test_loader)

9920512it [00:11, 864693.11it/s]                              



Test set: Alpha 1, Average loss: 0.1174, Accuracy: 9660/10000 (97%)



In [8]:
# A pretrinaed Network with a branch parrallel to FC1 for the purpose of redcucing from 128 to 64 neurons
# See also image 'highlevel.png'
class PrunedNet(nn.Module):
    def __init__(self,pretrainedmodel):
        super(PrunedNet, self).__init__()
        self.pretrainedNet = pretrainedmodel
        
        # No need to update weights of the pretrained network        
        for param in self.pretrainedNet.parameters():
            param.requires_grad=False
            
        # In case the weights of the pretrained network are also desired to be updated then 
        # comment out the above 2 lines of code
        
        # Branch parallel to the fc1 layer in the original pre-trained network
        self.fcc = nn.Linear(128, 64 , bias = False)
        self.fcd = nn.Linear(64, 128, bias = False)
        
    def forward(self, x, alpha=0.1):
        x = self.pretrainedNet.conv1(x)
        x = self.pretrainedNet.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv2(x)
        x = self.pretrainedNet.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv3(x)
        x = self.pretrainedNet.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        xorg = torch.flatten(x, 1)
        x1 = self.pretrainedNet.fc1(xorg)
        
        x2 = self.fcd((F.relu(self.fcc(x1))))

        x = alpha * F.relu(x1) + (1-alpha) * x2

        
        x = self.pretrainedNet.fc2(x)

        x= F.relu(x)
        
        x = self.pretrainedNet.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
pruned_model = PrunedNet(model).to(device)

In [18]:
# Check to see if the original branch(alpha = 1 ) gives the same accuracy.
# If not, then something wrong with the implementation
print('Original accuracy:')
test(model, device, test_loader)
print('Accuracy of pruned_model at alpha = 1:')
print(test(pruned_model, device, test_loader, alpha = 1))

Original accuracy:


9920512it [47:24, 3487.12it/s]   



Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9660/10000 (97%)

Accuracy of pruned_model at alpha = 1:


9920512it [47:26, 3485.51it/s]   



Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9660/10000 (97%)

None


In [10]:
# 2 For loops, The outer is for varying the alpha values. The inner loop are the training epochs
# The schedule of how alpha is decayed is something to explore.
# In this case, a simple, linear decay has been used.
optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, pruned_model.parameters()), lr=1.0)
for alpha in np.arange(0.9,-0.05,-0.1):
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, 10 + 1):
        train(10,pruned_model, device, train_loader, optimizer, epoch, alpha)
        test(pruned_model, device, test_loader, alpha)
        scheduler.step()

Train Epoch: 1, alpha: 0.9, [0/60000 (0%)]	Loss: 0.126012
Train Epoch: 1, alpha: 0.9, [640/60000 (1%)]	Loss: 0.099861
Train Epoch: 1, alpha: 0.9, [1280/60000 (2%)]	Loss: 0.030259
Train Epoch: 1, alpha: 0.9, [1920/60000 (3%)]	Loss: 0.113805
Train Epoch: 1, alpha: 0.9, [2560/60000 (4%)]	Loss: 0.056267
Train Epoch: 1, alpha: 0.9, [3200/60000 (5%)]	Loss: 0.132351
Train Epoch: 1, alpha: 0.9, [3840/60000 (6%)]	Loss: 0.044281
Train Epoch: 1, alpha: 0.9, [4480/60000 (7%)]	Loss: 0.194816
Train Epoch: 1, alpha: 0.9, [5120/60000 (9%)]	Loss: 0.181616
Train Epoch: 1, alpha: 0.9, [5760/60000 (10%)]	Loss: 0.024821
Train Epoch: 1, alpha: 0.9, [6400/60000 (11%)]	Loss: 0.063201
Train Epoch: 1, alpha: 0.9, [7040/60000 (12%)]	Loss: 0.202237
Train Epoch: 1, alpha: 0.9, [7680/60000 (13%)]	Loss: 0.169817
Train Epoch: 1, alpha: 0.9, [8320/60000 (14%)]	Loss: 0.087185
Train Epoch: 1, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.064166
Train Epoch: 1, alpha: 0.9, [9600/60000 (16%)]	Loss: 0.062828
Train Epoch: 1, alpha

9920512it [01:46, 92760.98it/s]  



Test set: Alpha 0.9, Average loss: 0.1149, Accuracy: 9656/10000 (97%)

Train Epoch: 2, alpha: 0.9, [0/60000 (0%)]	Loss: 0.061004
Train Epoch: 2, alpha: 0.9, [640/60000 (1%)]	Loss: 0.124240
Train Epoch: 2, alpha: 0.9, [1280/60000 (2%)]	Loss: 0.093297
Train Epoch: 2, alpha: 0.9, [1920/60000 (3%)]	Loss: 0.067250
Train Epoch: 2, alpha: 0.9, [2560/60000 (4%)]	Loss: 0.014999
Train Epoch: 2, alpha: 0.9, [3200/60000 (5%)]	Loss: 0.039850
Train Epoch: 2, alpha: 0.9, [3840/60000 (6%)]	Loss: 0.015075
Train Epoch: 2, alpha: 0.9, [4480/60000 (7%)]	Loss: 0.071926
Train Epoch: 2, alpha: 0.9, [5120/60000 (9%)]	Loss: 0.135823
Train Epoch: 2, alpha: 0.9, [5760/60000 (10%)]	Loss: 0.048122
Train Epoch: 2, alpha: 0.9, [6400/60000 (11%)]	Loss: 0.051579
Train Epoch: 2, alpha: 0.9, [7040/60000 (12%)]	Loss: 0.055190
Train Epoch: 2, alpha: 0.9, [7680/60000 (13%)]	Loss: 0.028332
Train Epoch: 2, alpha: 0.9, [8320/60000 (14%)]	Loss: 0.052554
Train Epoch: 2, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.050594
Train Epoch

9920512it [01:57, 84563.79it/s]  



Test set: Alpha 0.9, Average loss: 0.1150, Accuracy: 9654/10000 (97%)

Train Epoch: 3, alpha: 0.9, [0/60000 (0%)]	Loss: 0.027972
Train Epoch: 3, alpha: 0.9, [640/60000 (1%)]	Loss: 0.069959
Train Epoch: 3, alpha: 0.9, [1280/60000 (2%)]	Loss: 0.073368
Train Epoch: 3, alpha: 0.9, [1920/60000 (3%)]	Loss: 0.029458
Train Epoch: 3, alpha: 0.9, [2560/60000 (4%)]	Loss: 0.110904
Train Epoch: 3, alpha: 0.9, [3200/60000 (5%)]	Loss: 0.041146
Train Epoch: 3, alpha: 0.9, [3840/60000 (6%)]	Loss: 0.065918
Train Epoch: 3, alpha: 0.9, [4480/60000 (7%)]	Loss: 0.144103
Train Epoch: 3, alpha: 0.9, [5120/60000 (9%)]	Loss: 0.063715
Train Epoch: 3, alpha: 0.9, [5760/60000 (10%)]	Loss: 0.044290
Train Epoch: 3, alpha: 0.9, [6400/60000 (11%)]	Loss: 0.082962
Train Epoch: 3, alpha: 0.9, [7040/60000 (12%)]	Loss: 0.028821
Train Epoch: 3, alpha: 0.9, [7680/60000 (13%)]	Loss: 0.085409
Train Epoch: 3, alpha: 0.9, [8320/60000 (14%)]	Loss: 0.040575
Train Epoch: 3, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.074253
Train Epoch

...


Test set: Alpha 2.220446049250313e-16, Average loss: 0.1636, Accuracy: 9627/10000 (96%)

Train Epoch: 9, alpha: 2.220446049250313e-16, [0/60000 (0%)]	Loss: 0.000535
Train Epoch: 9, alpha: 2.220446049250313e-16, [640/60000 (1%)]	Loss: 0.031635
Train Epoch: 9, alpha: 2.220446049250313e-16, [1280/60000 (2%)]	Loss: 0.147797
Train Epoch: 9, alpha: 2.220446049250313e-16, [1920/60000 (3%)]	Loss: 0.005442
Train Epoch: 9, alpha: 2.220446049250313e-16, [2560/60000 (4%)]	Loss: 0.036508
Train Epoch: 9, alpha: 2.220446049250313e-16, [3200/60000 (5%)]	Loss: 0.009525
Train Epoch: 9, alpha: 2.220446049250313e-16, [3840/60000 (6%)]	Loss: 0.026584
Train Epoch: 9, alpha: 2.220446049250313e-16, [4480/60000 (7%)]	Loss: 0.034567
Train Epoch: 9, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.034555
Train Epoch: 9, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.064914
Train Epoch: 9, alpha: 2.220446049250313e-16, [6400/60000 (11%)]	Loss: 0.017377
Train Epoch: 9, alpha: 2.220446049250313e-1

9920512it [21:38, 7641.87it/s]   



Test set: Alpha 2.220446049250313e-16, Average loss: 0.1653, Accuracy: 9625/10000 (96%)

Train Epoch: 10, alpha: 2.220446049250313e-16, [0/60000 (0%)]	Loss: 0.080274
Train Epoch: 10, alpha: 2.220446049250313e-16, [640/60000 (1%)]	Loss: 0.064881
Train Epoch: 10, alpha: 2.220446049250313e-16, [1280/60000 (2%)]	Loss: 0.009796
Train Epoch: 10, alpha: 2.220446049250313e-16, [1920/60000 (3%)]	Loss: 0.039202
Train Epoch: 10, alpha: 2.220446049250313e-16, [2560/60000 (4%)]	Loss: 0.029928
Train Epoch: 10, alpha: 2.220446049250313e-16, [3200/60000 (5%)]	Loss: 0.010500
Train Epoch: 10, alpha: 2.220446049250313e-16, [3840/60000 (6%)]	Loss: 0.051976
Train Epoch: 10, alpha: 2.220446049250313e-16, [4480/60000 (7%)]	Loss: 0.070426
Train Epoch: 10, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.058179
Train Epoch: 10, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.183544
Train Epoch: 10, alpha: 2.220446049250313e-16, [6400/60000 (11%)]	Loss: 0.031404
Train Epoch: 10, alpha: 2.220446

9920512it [21:48, 7580.42it/s]   



Test set: Alpha 2.220446049250313e-16, Average loss: 0.1640, Accuracy: 9619/10000 (96%)



In [16]:
# Test to see the performance of the parallel branch by setting alpha = 0
# If accuracy is still high(approx. equal to when alpha =1),
# then it hints that the parrallel branch by itself is enough.
# Hence, the original pretrained model can then be pruned to a smaller size
test(pruned_model, device, test_loader, 0 )
test(pruned_model, device, test_loader, 1 )

9920512it [34:18, 4819.88it/s]   



Test set: Alpha 0, Average loss: 0.1640, Accuracy: 9619/10000 (96%)



9920512it [34:19, 4816.72it/s]   



Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9660/10000 (97%)



In [12]:
# Final pruned model after doing the appropriate matrix(or tensor) multiplications
# Note that FC1 output has now only 64 neurons instead of 128
# Likewise, the input of FC2 has 64 instead of 128 neurons
class SmallerNet(nn.Module):
    def __init__(self):
        super(SmallerNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(256 * 1 * 1, 64, bias =False)
        self.fc2 = nn.Linear(64, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        # Note that alpha is not being used here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
smaller_model = SmallerNet().to(device)

In [13]:
# Since the model is randomly initilaized, the accuracy is low
test(smaller_model, device, test_loader, 0 )

9920512it [23:50, 6936.85it/s]   



Test set: Alpha 0, Average loss: 2.3021, Accuracy: 928/10000 (9%)



In [14]:
# The pretrained weights are loaded after the appropriate matrix multiplications
from copy import deepcopy
smaller_model_dict= deepcopy(model.state_dict())
print(smaller_model_dict["fc1.weight"].shape)
print(smaller_model_dict["fc2.weight"].shape)
smaller_model_dict["fc1.weight"] = pruned_model.state_dict()["fcc.weight"] @ pruned_model.state_dict()["pretrainedNet.fc1.weight"]
smaller_model_dict["fc2.weight"] = pruned_model.state_dict()["pretrainedNet.fc2.weight"] @ pruned_model.state_dict()["fcd.weight"] 
print(smaller_model_dict["fc1.weight"].shape)
print(smaller_model_dict["fc2.weight"].shape)
smaller_model_dict.keys()
smaller_model.load_state_dict(smaller_model_dict)

torch.Size([128, 256])
torch.Size([50, 128])
torch.Size([64, 256])
torch.Size([50, 64])


<All keys matched successfully>

In [15]:
# Test accuracy of the smaller_model is the same as pruned_model when alpha=0
# Note that only pruned_model needs alpha values not the smaller_model(Check the forward functions for both)
# However, for the sake of simplicity, test() and train() funcitons were defined assuming that
# alpha is needed. But this alpha when passed to the smaller_model is irrelevant.
# TODO: Reimplement the train() and test() functions such that alpha is only required for pruned_model
test(smaller_model, device, test_loader, 0 )

9920512it [23:55, 6909.00it/s]   



Test set: Alpha 0, Average loss: 0.1640, Accuracy: 9619/10000 (96%)



In [102]:
print(smaller_model.state_dict()["fc1.weight"].shape)
print(smaller_model.state_dict()["fc2.weight"].shape)
print("--"*10)
print(model.state_dict()["fc1.weight"].shape)
print(model.state_dict()["fc2.weight"].shape)
print("--"*10)
print(pruned_model.state_dict()["pretrainedNet.fc1.weight"].shape)
print(pruned_model.state_dict()["fcc.weight"].shape)
print((pruned_model.state_dict()["fcc.weight"] @ pruned_model.state_dict()["pretrainedNet.fc1.weight"]).shape)
print("--"*10)
print(pruned_model.state_dict()["pretrainedNet.fc2.weight"].shape)
print(pruned_model.state_dict()["fcd.weight"].shape)

torch.Size([64, 256])
torch.Size([50, 64])
--------------------
torch.Size([128, 256])
torch.Size([50, 128])
--------------------
torch.Size([128, 256])
torch.Size([64, 128])
torch.Size([64, 256])
--------------------
torch.Size([50, 128])
torch.Size([128, 64])


In [71]:
pruned_model.state_dict()["fcc.weight"].shape
pruned_model.state_dict()["pretrainedNet.fc1.weight"]

torch.Size([64, 128])

In [32]:
class PrunedNet(nn.Module):
    def __init__(self,pretrainedmodel):
        super(PrunedNet, self).__init__()
        self.pretrainedNet = pretrainedmodel
        
        # No need to update weights of the pretrained network        
        for param in self.pretrainedNet.parameters():
            param.requires_grad=False
        
        self.fca = nn.Linear(256, 30, bias = False)
        self.fcb = nn.Linear(30, 128, bias = False)
        
        self.fcc = nn.Linear(128, 10, bias = False)
        self.fcd = nn.Linear(10, 50, bias = False)
        
    def forward(self, x, alpha=0.9):
        x = self.pretrainedNet.conv1(x)
        x = self.pretrainedNet.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv2(x)
        x = self.pretrainedNet.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv3(x)
        x = self.pretrainedNet.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        xorg = torch.flatten(x, 1)
        x1 = self.pretrainedNet.fc1(xorg)
        x2 = self.fca(xorg)
        x2 = self.fcb(x2)
        #x = alpha * x1 + (1-alpha) * x2
        xorg = F.relu(x2)
        
        #x1 = self.pretrainedNet.fc2(xorg)
        x2 = self.fcc(xorg)
        x2 = self.fcd(x2)
        #x = alpha * x1 + (1-alpha) * x2
        x= F.relu(x2)
        
        x = self.pretrainedNet.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
pruned_model = PrunedNet(model).to(device)

8

In [26]:
for p in pruned_model.pretrainedNet.parameters():
    pass
p.requires_grad

True

In [ ]:
class smallerNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 1 * 1, 64, bias =False)
        self.fc2 = nn.Linear(64, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
smaller_model = smallerNet().to(device)

In [11]:
layer3 = model_dict['conv3.weight'].cpu().numpy()
num_filters = layer3.shape[0]

In [12]:
layer3.shape

(256, 2, 3, 3)

In [13]:
# class PrunedNet(nn.Module):
#     def __init__(self,pretrainedmodel):
#         super(PrunedNet, self).__init__()
#         self.pretrainedNet = pretrainedmodel
        
#         # No need to update weights of the pretrained network        
#         for param in self.pretrainedNet.parameters():
#             param.requires_grad=False
        
#         self.fca = nn.Linear(256, 30, bias = False)
#         self.fcb = nn.Linear(30, 128, bias = False)
        
#         self.fcc = nn.Linear(128, 10, bias = False)
#         self.fcd = nn.Linear(10, 50, bias = False)
        
#     def forward(self, x, alpha=0.9):
#         x = self.pretrainedNet.conv1(x)
#         x = self.pretrainedNet.bn1(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.pretrainedNet.conv2(x)
#         x = self.pretrainedNet.bn2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.pretrainedNet.conv3(x)
#         x = self.pretrainedNet.bn3(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
        
#         xorg = torch.flatten(x, 1)
#         x1 = self.pretrainedNet.fc1(xorg)
#         x2 = self.fca(xorg)
#         x2 = self.fcb(x2)
#         x = alpha * x1 + (1-alpha) * x2
#         xorg = F.relu(x)
        
#         x1 = self.pretrainedNet.fc2(xorg)
#         x2 = self.fcc(xorg)
#         x2 = self.fcd(x2)
#         x = alpha * x1 + (1-alpha) * x2
#         x= F.relu(x)
        
#         x = self.pretrainedNet.fc3(x)
#         output = F.log_softmax(x, dim=1)
#         return output
# pruned_model = PrunedNet(model).to(device)

256

In [14]:
scores = np.sum(np.sum(np.sum(np.abs(layer3),3),2),1)
print(scores)

[2.8458521 2.4906092 2.182644  2.4704623 2.604546  2.7074933 2.4273744
 2.4395137 2.7046404 2.2236407 2.724819  2.346136  2.4736838 2.704204
 2.352558  1.8051548 2.0248778 2.1634579 2.0994892 2.1173074 2.4371142
 2.56626   2.1074808 2.8850985 2.6320398 2.3986905 2.6123188 2.4185576
 2.3762336 2.3464718 2.3099728 2.3896155 2.3797746 2.571484  2.682601
 2.457549  2.5476065 2.703555  2.171604  2.8341658 2.7164273 2.3056388
 2.2262316 2.4162507 2.162299  2.1605859 2.3533275 2.154541  2.29719
 2.3968291 2.1048923 2.3813574 2.5073893 2.5824394 2.5691903 2.173055
 2.6351976 2.549543  2.6614966 2.5639572 2.6166143 2.8284776 2.1105638
 2.4044895 2.4763665 2.1381226 2.629759  2.30014   2.2500648 2.2014756
 1.8681186 1.7940662 2.735925  2.3465033 2.6203418 2.2923179 2.656416
 2.6224637 2.6273053 2.1582756 2.4212275 1.8324049 2.1633458 2.0751681
 2.4310641 2.3555272 2.2806036 2.3307126 2.0461018 2.3769479 2.0755684
 2.1711507 2.357153  2.3072407 2.2286    2.5354738 2.273097  2.405572
 2.4387927 2.

In [15]:
ordered_indices = np.argsort(scores)
ordered_indices

array([222,  71, 124,  15,  81, 179,  70, 229, 173, 123, 217, 119, 180,
       129, 161, 238, 213,  16, 206, 187, 105,  88, 139, 197, 209,  83,
        90, 131, 214, 177,  18, 166,  50, 143,  22,  62, 137, 127, 210,
        19, 133,  65, 171, 220, 208, 245, 144, 184,  47, 215,  79,  45,
        44,  82,  17,  91,  38, 150,  55, 128,   2, 120, 151, 191,  69,
       130, 159, 252, 181,   9,  42,  94, 227, 196, 231, 236, 247,  68,
        99, 138, 182, 116, 218,  96, 167,  86, 234, 235,  75, 233,  48,
        67,  41,  93,  30, 100, 107, 110, 135, 189,  87, 140, 154, 163,
        11,  29,  73, 205, 242,  14,  46, 125, 237,  85, 115,  92, 240,
       230, 228, 103, 142, 108,  28,  89, 145,  32,  51, 147, 255, 165,
       226, 212,  31,  49,  25, 117, 188,  63,  97, 168, 169, 104, 241,
       194,  43,  27,  80, 244,   6,  84, 109,  20, 232,  98,   7, 162,
       134, 198,  35, 126, 155, 132,   3, 114,  12,  64, 192, 246, 136,
       219, 221, 249,   1, 195,  52, 160, 172, 253, 122, 248, 20

In [16]:
num_filters_to_retain = int(num_filters * (1 - remove_ratio))
filters_idxs_to_retain = ordered_indices[:num_filters_to_retain]

In [17]:
model_dict['conv3.weight'][filters_idxs_to_retain].shape

torch.Size([192, 2, 3, 3])

In [18]:
model_dict['conv3.weight'] = model_dict['conv3.weight'][filters_idxs_to_retain]
model_dict["bn3.weight"] = model_dict['bn3.weight'][filters_idxs_to_retain]
model_dict["bn3.running_mean"] = model_dict['bn3.running_mean'][filters_idxs_to_retain]
model_dict["bn3.running_var"] = model_dict['bn3.running_var'][filters_idxs_to_retain]
model_dict["bn3.bias"] = model_dict['bn3.bias'][filters_idxs_to_retain]


In [19]:
model_dict["fc1.weight"] = model_dict["fc1.weight"][:,filters_idxs_to_retain]

In [20]:
#print(model_dict["fc1.weight"].shape)
#print(model_dict["fc1.weight"][:,filters_idxs_to_retain].shape)


In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 192, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(192)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(192 * 1 * 1, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
model = Net().to(device)

In [27]:
model.load_state_dict(model_dict)


<All keys matched successfully>

In [28]:
test(model, device, test_loader)


Test set: Average loss: 0.3935, Accuracy: 8718/10000 (87%)



In [29]:
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 10 + 1):
    train(10,model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.204891
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.123856
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.205159
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.117286
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.038898
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.289284
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.196593
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.238233
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.399758
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.115703
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.118389
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.515954
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.176568
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.185592
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.064054
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.101278
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.185965
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.115036
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.062826
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.106442
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.078401
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.059897
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.054985
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.357186
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.062702
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.161579
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.172899
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.123033
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.111696
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.147828
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.193112
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.046242
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.065079
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.226090
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.143976
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.107194
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.069249
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.118811
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.031427
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.037398


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.004859
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.296020
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.071183
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.029004
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.087584
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.138139
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.175175
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.055548
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.197740
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.010896
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.051935
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.009342
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.217856
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.067858
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.081472
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.093157
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.087819
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.129756
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.073688
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.134740


Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.082824
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.102972
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.052483
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.074387
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.156844
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.004268
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.046857
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.047494
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.017852
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.074836
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.013387
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.055583
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.110941
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.008111
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.010436
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.075385
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.070993
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.130795
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.121007
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.018806


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.006007
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.056902

Test set: Average loss: 0.1118, Accuracy: 9671/10000 (97%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.092088
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.093092
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.055554
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.061812
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.085422
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.003094
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.127153
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.020580
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.156190
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.026704
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.036243
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.025266
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.057215
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.018690
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.043181
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.058149
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.059936
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.024038
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.203296
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.006788
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.131806
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.034274
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.014773
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.015448
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.049023
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.003238
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.023971
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.032843
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.009613
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.024686
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.032912
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.025528
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.370933
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.131681
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.043389
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.066766
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.117187


In [23]:
torch.save(model, "pruned_model.pth")

/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " 

In [24]:
new_model = torch.load("pruned_model.pth")

In [25]:
test(new_model, device, test_loader)


Test set: Average loss: 0.1149, Accuracy: 9673/10000 (97%)



In [26]:
import numpy as np
import torch
from torch.autograd import Variable

x = Variable(torch.Tensor([5]),requires_grad=True)
w0 = Variable(torch.Tensor([2]),requires_grad=False)

w1 = Variable(torch.Tensor([3]),requires_grad=False)


y1 = x**2*w0
y2 = y1*w1


gradients = []
y1.register_hook(lambda x: gradients.append(x))
y2.register_hook(lambda x: gradients.append(x))
x.register_hook(lambda x: gradients.append(x))

y2.backward()

gradients

[tensor([1.]), tensor([3.]), tensor([60.])]

In [27]:
def print_value(value):
    print("The value is {}".format(value.item()))

x = Variable(torch.Tensor([5]),requires_grad=True)
w0 = Variable(torch.Tensor([2]),requires_grad=False)

w1 = Variable(torch.Tensor([3]),requires_grad=False)


y1 = x**2*w0
y2 = y1*w1


gradients = []
y1.register_hook(lambda x: print_value(x))
y2.register_hook(lambda x: print_value(x))
x.register_hook(lambda x: print_value(x))



In [28]:
y2.backward()

The value is 1.0
The value is 3.0
The value is 60.0
